In [1]:
import os
%pwd

'e:\\Food_Delivery\\notebooks'

In [2]:
os.chdir('../')

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_path:Path

In [4]:
from src.constants import *
from src.utils import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path=config.model_path
        )

        return model_trainer_config

In [6]:
import os
from src.logging import logger
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import pickle

from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

scaler=StandardScaler()


In [7]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
        
    def initiate_model_training(self):
        data = pd.read_csv(self.config.data_path)
        X = data.drop('Time_taken(min)', axis = 1)
        y = data['Time_taken(min)'] 
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
        
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        
        
        rf = RandomForestRegressor()
        
        rf.fit(X_train, y_train)
        
        rf_model = self.config.model_path
        
        with open(rf_model, 'wb') as file:
            pickle.dump(rf, file)
        

In [8]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config = model_trainer_config)
    model_trainer.initiate_model_training()
except Exception as e:
    raise e

[2023-08-21 14:24:50,985: INFO: __init__: yaml_file: config\config.yaml loaded succesfully...]
[2023-08-21 14:24:50,987: INFO: __init__: yaml_file: params.yaml loaded succesfully...]
[2023-08-21 14:24:50,989: INFO: __init__: Directory created at: artifacts]
[2023-08-21 14:24:50,989: INFO: __init__: Directory created at: artifacts/models]
